# Multi-hospital Organ Transplants

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent directory, **MultipleAgents**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

<font color='red'>IMPORTANT NOTE: you should run **AgentsAndEvents.ipynb** prior to running this notebook to ensure that **AE.pl** contains up to date predicates! You must run the notebook, not just update and save it.</font>

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

## Timely communication

Both our individual hospitals and **monSys** may have expectations about timely communication with other agents. 

When agent A sends a message to agent B, agent A might expect that they will receive a response back from agent B within a certain time frame, whether a simple message of acknowledgement or a message containing new information. An agent may well wish to know that their message has been received and somehow processed by the desired recipient, and may also expect that communication chains will proceed in reasonable time. For instance, if **monSys** sends a request to a hospital, asking for a patient's medical history, and **monSys** needs the medical history before it can approve a patient for an organ transplant, **monSys** may want the medical history to be shared quite quickly, so there are no unnecessary delays. Reducing delays would both reduce the amount of backlog agents have to process, and is also particularly vital in our medical setting, where there is a risk that patients may become more unwell or pass away if they are forced to wait extended periods of time for appropriate medical treatment.

When an agent relies on information provided by other agents, it might expect regular updates in order to be more confident that information used to make important decisions is not out of date.

If an agent's expectation about timely communication is violated, this could prompt the agent to send a follow up message to the agent that it is waiting on, asking for their earlier message to be addressed as soon as possible.

Example expectation rules are created for different subcategories of "Timely communicaton" below. Narrative events are created at the end of the notebook to exhibit testing of these rules.

### Regular updates

An agent might expect that they are kept up to date on important information which is provided by other agents. In this case, **monSys** expects that the hospital which has added a patient to the organ waiting list will provide updates about the patient's information at least every 5 time periods, starting when the patient is accepted, up until the patient is removed from the waiting list. Note that while this is not currently encoded, we would also want to include a succesful transplant notification as a suitable event to fulfil our expectation. 

In reality, the maximum time allowed between patient updates might be 6 months, but for testing purposes, the time period is kept low here.

In [2]:
% File: expWaitListUpdates.pl

% MonSys expect that the relevant hospital will update a patient's information or the patient will 
% be removed from the waiting list within 5 time periods of them being added or an update provided.
initially(monSys:exp_rule(or([happ(send(Hospital, waitAccept(ID))), happ(receive(Hospital, 
waitUpdate(ID,_,_)))]), withinStartNext(or([happ(receive(Hospital, waitUpdate(ID,_,_))), 
happ(receive(Hospital, waitDelReq(ID, _)))]), 5), dependent, 
"Regular updates":"A hospital (not Otago) will update a waiting recipient's information at least every 5 time periods")).


We can also apply further conditions to our expectation rule. We could tweak it to say that **monSys** expects updates from Otago Hospital every 3 time periods, and updates from other hospitals every time periods, as shown below. Perhaps **monSys** wants to keep a closer eye on patients from a newly established hospital, or one that has had past issues with record keeping. It would also be possible to encode different update timings for different organs, although this would begin to get more complicated if one waiting record is still used for a patient waiting for multiple organs.

In [3]:
% File: expWaitListUpdates.pl

% MonSys expect that the Otago hospital will update a patient's information or the patient will 
% be removed from the waiting list within 3 time periods of them being added or an update provided.
initially(monSys:exp_rule(and([or([happ(send(Hospital, waitAccept(ID))), happ(receive(Hospital, 
waitUpdate(ID,_,_)))]), (condition(Hospital == "Otago"))]), withinStartNext(or([happ(receive(Hospital, waitUpdate(ID,_,_))), 
happ(receive(Hospital, waitDelReq(ID, _)))]), 3), dependent, 
"Regular updates":"Otago will update a waiting recipient's information at least every 3 time periods")).

% MonSys expect that the relevant hospital will update a patient's information or the patient will 
% be removed from the waiting list within 5 time periods of them being added or an update provided.
initially(monSys:exp_rule(and([or([happ(send(Hospital, waitAccept(ID))), happ(receive(Hospital, 
waitUpdate(ID,_,_)))]), (condition(Hospital \= "Otago"))]), withinStartNext(or([happ(receive(Hospital, waitUpdate(ID,_,_))), 
happ(receive(Hospital, waitDelReq(ID, _)))]), 5), dependent, 
"Regular updates":"A hospital (not Otago) will update a waiting recipient's information at least every 5 time periods")).


While updates could be used to add details of new medical tests performed on a patient, or correct past information entered in error, for simplicity, we declare the behaviour of wait list updates to simply overwrite the previous organ and detail information in a patient's waiting list record.

In [4]:
% File waitUpdate.pl
terminates(monSys:receive(Hospital, waitUpdate(ID, _, _)), waiting(ID, Hospital, _, _, T1), T):- T1 < T.
initiates(monSys:receive(Hospital, waitUpdate(ID, Organs, Details)), waiting(ID, Hospital, Organs, Details, T1), T):- holdsAt(monSys, waiting(ID, Hospital, _, _, T1),T).

### Timely response / action

Here **monSys** expects that once it receives a donation offer, the offer will be matched or a declaration that no match was found will be made at the next time period. Here, **monSys** is not expecting timely action from another hospital, but rather from whoever is in charge of making matches internally in **monSys**.

In [5]:
% File: donationExpectations.pl
% Donation offer to match expectation
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(ID, Organs, _))), 
within(or([happ(noMatch(Hospital, ID)), happ(match(_, Hospital, ID, _, _, _, _, _))]),1), 
dependent, "Timely response":"A donation offer should be matched or a declaration of no match found will occur in the next time period")).

**monSys** also expects that once it offers a match to the donor's hospital, the recipient's hospital, and the transplant location, **monSys** will receive a message within 3 time periods from each of these agents accepting the match.

This expectation could be altered to allow for match re-negotiations. For example, if the match included a suggested time for the transplant to take place, the hospitals could ask for a later time to allow for transport considerations or to reduce the load on a given hospital. The scenario could also be extended further to allow for acceptance under certain conditions. For instance, the recipient's hospital might accept the match on the condition that the donor returns a negative test for COVID 19. Fluents of this level of detail are not covered here.

In [6]:
% File: matchAcceptance.pl
initially(monSys:exp_rule(happ(send(DonorHospital, recipientFound(MatchID, _, _, _, _, _))),
within(happ(receive(DonorHospital, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the donor's hospital within 3 time periods")).
initially(monSys:exp_rule(happ(send(RecipientHospital, donorFound(MatchID, _, _, _, _, _))),
within(happ(receive(RecipientHospital, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the recipient's hospital within 3 time periods")).
initially(monSys:exp_rule(happ(send(Location, locationSelected(MatchID, _, _, _, _))),
within(happ(receive(Location, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the transplant location within 3 time periods")).

### Outcome notification

There are many cases where an agent instructs another agent to perform a given action, and wants to be informed of the outcome. 

Hospitals and the monitoring system both want to receive updates on whether a transplant was successful. Individual hospitals may have expectations about the percentage of organs they donate which lead to successful transplants, or about knowing whether their own patient has been successfully treated. The monitoring system may have the capability to calculate system wide statistics on successful transplants, in which case it would expect to be notified whether transplants were successful.

Below, the expectation rules declared state that **monSys** and the donor and recipient hospitals will be notified of the transplant outcome within 5 time periods of the match being confirmed (for **monSys**) or within 5 time periods of being notified of the match confirmation (for the individual hospitals).

In [7]:
% File: followUp.pl
initially(monSys:exp_rule(and([happ(confirmationMatch(ID)), role(Location, MatchID, transplant)]),
within(happ(receive(Location, transplantOutcome(ID, _))),5),
dependent,"Outcome notification":"monSys will be notified of transplant outcome within X time periods")).

initially(Agent:exp_rule(and([happ(receive(confirmationNotification(ID))), 
role(Location, MatchID, transplant), self(Self), or([role(Self, matchID, donor), 
role(Self, MatchID, recipient)])]), within(happ(receive(Location, transplantOutcome(ID, _))),5), dependent,
"Outcome notification":"The donor and recipient hospitals should be notified of transplant outcome within X time periods")):- 
Agent \= monSys.

### Test Narrative

At time period 0, Otago hospital sends a waiting list add request to **monSys** for patient 104. This patient is in need of a heart, and is blood type A.

At time period 4, Otago hospital sends a waiting list update to **monSys** for patient 104, which adds the information that the patient has asthma.

At time period 5, Christchurch hospital sends a waiting list add request for patient 105, who needs a liver and is blood type B.

At time period 7, Christchurch hospital sends a request asking that patient 105 be deleted from the waiting list, due to their death.

At time period 10, Wellington hospital sends a donation offer to **monSys** for patient 201. They have five organs suitable for donation, and are blood type A.

At time period 12, **monSys** matches the waiting list record for patient 104 from Otago to the donor offer from patient 201 from Wellington.

In [8]:
% File: waitListAdditions.pl


happensAtNarrative("Otago":send(monSys, waitAddReq(104, ["heart"], [bloodType:"A"])),0).

happensAtNarrative("Otago":send(monSys, waitUpdate(104, ["heart"], [bloodType:"A", condition:"Asthma"])), 4).
happensAtNarrative("Christchurch":send(monSys, waitAddReq(105, ["liver"], [bloodType:"B"])),5).
happensAtNarrative("Christchurch":send(monSys, waitDelReq(105, "Patient Death")), 7).
happensAtNarrative("Wellington": send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], [bloodType:"A"])),10).
happensAtNarrative(monSys:match(701, "Otago", 104, "Wellington", 201, "Auckland", ["heart"], []),12).


We set up our environment for 20 time periods, so that we can query events and fluents in this time frame.

In [9]:
?- run(20).

true.

At time period 1, **monSys** receives Otago's waiting list add request for patients 104.
At time period 2, **monSys** sends Otago a message notifying the hospital that patient 104 has been successfully added to the waiting list.
At time period 2, **monSys** expects that Otago will provide an update on patient 104 within 3 time periods.

In [10]:
?- T = 1, happensAt(monSys, Event, T).
?- T = 2, happensAt(monSys, Event, T).
?- T= 2, holdsAt(monSys, exp(_,_,_,Condition,_,Message), T).


T = 1, Event = receive(b'Otago', waitAddReq(104, [b'heart'], [Functor(188685,2,bloodType,b'A')])) .
T = 2, Event = send(b'Otago', waitAccept(104)) .
T = 2, Condition = withinStartNext(or([Functor(14512269,1,receive(b'Otago', waitUpdate(104, _1850, _1852))), Functor(14512269,1,receive(b'Otago', waitDelReq(104, _1880)))]), 3), Message = :(b'Regular updates', b"Otago will update a waiting recipient's information at least every 3 time periods") .

By time period 4, **monSys** expects that information for patient 104 will be updated before the next time period is up.

In [11]:
?- holdsAt(monSys, exp(_, _, _, within(_, TimeRemaining), _, Message), 4).

TimeRemaining = 1, Message = :(b'Regular updates', b"Otago will update a waiting recipient's information at least every 3 time periods") .

At time period 5, **monSys** receives the waiting list update from Otago hospital for patient 104. **monSys**' expectation that an update will occur is fulfilled.

In [12]:
?- happensAt(monSys, Event, 5).

Event = fulf(and([Functor(753805,1,[Functor(14512269,1,send(b'Otago', waitAccept(104))), Functor(14512269,1,receive(b'Otago', waitUpdate(104, _1702, _1704)))]), Functor(14487693,1,==(b'Otago', b'Otago'))]), withinStartNext(or([Functor(14512269,1,receive(b'Otago', waitUpdate(104, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,b'Asthma')]))), Functor(14512269,1,receive(b'Otago', waitDelReq(104, _1800)))]), 3), 2, within(or([Functor(14512269,1,receive(b'Otago', waitUpdate(104, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,b'Asthma')]))), Functor(14512269,1,receive(b'Otago', waitDelReq(104, _1800)))]), 0), dependent, :(b'Regular updates', b"Otago will update a waiting recipient's information at least every 3 time periods")) ;
Event = receive(b'Otago', waitUpdate(104, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,b'Asthma')])) .

As a new update event has occurred, a new expectation is created which says that another update will occur within another 3 time periods or the patient will have been deleted from the waiting list.

In [13]:
?- T= 5, holdsAt(monSys, exp(_,_,_,withinStartNext(Outcome, 3),_,Message), T).

T = 5, Outcome = or([Functor(14512269,1,receive(b'Otago', waitUpdate(104, _1934, _1936))), Functor(14512269,1,receive(b'Otago', waitDelReq(104, _1964)))]), Message = :(b'Regular updates', b"Otago will update a waiting recipient's information at least every 3 time periods") .

At time period 5, the waiting list update has not yet been processed, however, by time period 6 the waiting list record has been updated:

In [14]:
?- T = 5, holdsAt(monSys, waiting(ID, Hospital, Organs, Details, Time), T).
?- T = 6, holdsAt(monSys, waiting(ID, Hospital, Organs, Details, Time), T).

T = 5, ID = 104, Hospital = b'Otago', Organs = [ b'heart' ], Details = [ bloodType:b'A' ], Time = 1 .
T = 6, ID = 104, Hospital = b'Otago', Organs = [ b'heart' ], Details = [ bloodType:b'A', condition:b'Asthma' ], Time = 1 .

At time period 7, **monSys** accepts the waiting list add request for patient 105 from Christchurch. **monSys** expects that Christchurch hospital will provide an update on patient 105, or ask that they be removed from the list, within 5 time periods. (Recall that our expectation rules state that **monSys** expects updates within 3 time periods from Otago, and within 5 time periods from other hospitals.)

In [15]:
?- T = 7, happensAt(monSys, Event, T).
?- T= 7, holdsAt(monSys, exp(_,_,_,withinStartNext(Outcome, _),_,Message), T).

T = 7, Event = send(b'Christchurch', waitAccept(105)) .
T = 7, Outcome = or([Functor(14512269,1,receive(b'Christchurch', waitUpdate(105, _1894, _1896))), Functor(14512269,1,receive(b'Christchurch', waitDelReq(105, _1926)))]), Message = :(b'Regular updates', b"A hospital (not Otago) will update a waiting recipient's information at least every 5 time periods") .

At time period 8, **monSys** receives the waiting list deletion request for patient 105, and as such, the expectation associated with patient 105's waiting fluent is fulfilled.

In [16]:
?- T = 8, happensAt(monSys, Event, T), Event \= receive("Wellington", _).

T = 8, Event = fulf(and([Functor(753805,1,[Functor(14512269,1,send(b'Christchurch', waitAccept(105))), Functor(14512269,1,receive(b'Christchurch', waitUpdate(105, _1768, _1770)))]), Functor(14487693,1,\=(b'Christchurch', b'Otago'))]), withinStartNext(or([Functor(14512269,1,receive(b'Christchurch', waitUpdate(105, _1840, _1842))), Functor(14512269,1,receive(b'Christchurch', waitDelReq(105, b'Patient Death')))]), 5), 7, within(or([Functor(14512269,1,receive(b'Christchurch', waitUpdate(105, _1840, _1842))), Functor(14512269,1,receive(b'Christchurch', waitDelReq(105, b'Patient Death')))]), 4), dependent, :(b'Regular updates', b"A hospital (not Otago) will update a waiting recipient's information at least every 5 time periods")) ;
T = 8, Event = receive(b'Christchurch', waitDelReq(105, b'Patient Death')) .

At time period 9, the waiting list record for patient 104 has not been updated in 4 time periods, so a violation occurs.

In [17]:
?- T= 9, holdsAt(monSys, exp(_,_,_,within(_, TimeRemaining),_,Message), T).
?- happensAt(monSys, viol(_, _, _, _, _, Category:Message), T).

T = 9, TimeRemaining = -1, Message = :(b'Regular updates', b"Otago will update a waiting recipient's information at least every 3 time periods") .
Category = b'Regular updates', Message = b"Otago will update a waiting recipient's information at least every 3 time periods", T = 9 ;
Category = b'Timely response', Message = b'A donation offer should be matched or a declaration of no match found will occur in the next time period', T = 13 ;
Category = b'Timely response', Message = b"A match offer should be accepted by the donor's hospital within 3 time periods", T = 17 ;
Category = b'Timely response', Message = b"A match offer should be accepted by the recipient's hospital within 3 time periods", T = 17 ;
Category = b'Timely response', Message = b'A match offer should be accepted by the transplant location within 3 time periods', T = 17 .

At time period 11, **monSys** receives the donor offer message from Wellington hospital. This leads to the expectation that a `match` or a `noMatch` event will occur by the end of the next time period.

In [18]:
?- happensAt(monSys, Event, 11).
?- holdsAt(monSys, exp(_,_,_,within(_, TimeRemaining),_,Message), 11).

Event = receive(b'Wellington', donorOffer(201, [Atom('680197'), Atom('680325'), Atom('680453'), Atom('680581'), Atom('680709')], [Functor(188685,2,bloodType,b'A')])) .
TimeRemaining = 1, Message = :(b'Timely response', b'A donation offer should be matched or a declaration of no match found will occur in the next time period') .

In [19]:
?- happensAt(monSys, Event, 12).

Event = match(701, b'Otago', 104, b'Wellington', 201, b'Auckland', [b'heart'], []) .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>